In [101]:
from bokeh.models import ColumnDataSource, OpenURL, TapTool
from bokeh.models import Button, CustomJS, Div, TextInput
from bokeh.plotting import figure, output_file, save, show
from bokeh.events import Tap, ValueSubmit
from bokeh.models.callbacks import CustomJS
from bokeh.layouts import column, row
from bokeh.events import ButtonClick
import random
import os
import json
import numpy as np
import pandas as pd

In [102]:
f = open('data/name_to_link.json')
name_to_link = json.load(f)

f = open('data/clip_to_question.json')
clip_to_question = json.load(f)

pca_df = pd.read_csv("data/clip_q_to_tsne_q.csv")#"data/clip_q_to_pca_q.csv")
clip_q_to_pca = {row[0]: [row[1], row[2]] for index, row in pca_df.iterrows()}

In [103]:
AMOUNT = 1000
CIRCLE_SIZE = 30
IMAGE_SIZE = 26

TO_IGNORE= ['168b7587-f0e6-47b2-bb9b-831840f7eb8b_32351']
def check(clip_q):
    clip_f = clip_q[:clip_q.rfind("-")]
    if clip_f in TO_IGNORE:
        return False
    return True

clip_names = [clip_q for clip_q in clip_to_question if check(clip_q)][:AMOUNT]
x = [clip_q_to_pca[clip_q][0]/2 for clip_q in clip_names]
y = [clip_q_to_pca[clip_q][1]/2 for clip_q in clip_names]
frame_locations = [f"transparent_images/{clip_q[:clip_q.rfind('-')]}.png" for clip_q in clip_names]
original_frame_locations = [f"images/{clip_q[:clip_q.rfind('-')]}.jpg" for clip_q in clip_names]
clip_url = [name_to_link[f"{clip_q[:clip_q.rfind('-')]}.mp4"] for clip_q in clip_names]
clip_q = []
clip_a = []
clip_wa = []
clip_wb = []
clip_wc = []
clip_wd = []

for clip_n in clip_names:
    qa_string = clip_to_question[clip_n]
    qaw = qa_string.split("\n")
    q = qaw[0][qaw[0].find(":") + 2:]
    a = qaw[1][qaw[1].find(":") + 2:]
    wa = qaw[2][qaw[2].find(":") + 2:]
    wb = qaw[3][qaw[3].find(":") + 2:]
    wc = qaw[4][qaw[4].find(":") + 2:]
    wd = qaw[5][qaw[5].find(":") + 2:]
    
    clip_q.append(q)
    clip_a.append(a)
    clip_wa.append(wa)
    clip_wb.append(wb)
    clip_wc.append(wc)
    clip_wd.append(wd)

In [104]:
data = {'x':x,
        'y':y,
        'clip_names':clip_names,
        'url':frame_locations,
        'original_url':original_frame_locations,
        'clip_url':clip_url,
        'q':clip_q,
        'a':clip_a,
        'wa':clip_wa,
        'wb':clip_wb,
        'wc':clip_wc,
        'wd':clip_wd,
        'alpha':[1]*AMOUNT,
        }


source = ColumnDataSource(data)

f = open("html_txts/hover.txt", "r")
HOWER = f.read()

f = open("html_txts/js_code.txt", "r")
JS_CODE = f.read()

f = open("html_txts/search_code.txt", "r")
SEARCH_CODE = f.read()

p = figure(sizing_mode="stretch_both", 
           width=2500, 
           height=700, 
           tooltips=HOWER, 
           tools="pan,wheel_zoom,reset,tap,hover",)
p.outline_line_width = 3
p.outline_line_alpha = 0.5
p.outline_line_color = "navy"

p.xaxis.visible = False
p.yaxis.visible = False
p.xgrid.visible = False
p.ygrid.visible = False

circle = p.circle(x = 'x', y='y', fill_color="red", size=CIRCLE_SIZE, alpha='alpha', source = source,)
images = p.image_url(url = 'url', x = 'x', y='y',
                     anchor="center", w=IMAGE_SIZE, w_units="screen", 
                     h=IMAGE_SIZE, h_units="screen", dilate=True, alpha='alpha',
                     source = source, )

callback = CustomJS(args=dict(source=source), code=JS_CODE)
search_callback = CustomJS(args=dict(source=source, renderer=images), code=SEARCH_CODE)

source.selected.js_on_change('indices', callback)
p.hover.renderers = [circle]

text_input = TextInput(placeholder="Input a value and press Enter ...", width=1500, height = 50, css_classes = ["text-input-1223"])
text_input.sizing_mode = "stretch_width"
text_input.js_on_event(ValueSubmit, search_callback)

layout = column(p, text_input)
layout.sizing_mode = "stretch_both"

# save the results to a file
output_file(filename="plot_check.html", title="Static HTML file")
show(layout)

In [17]:
from bokeh.models import ColumnDataSource, OpenURL, TapTool
from bokeh.models import Button, CustomJS, Div, TextInput
from bokeh.plotting import figure, output_file, save, show
from bokeh.events import Tap, ValueSubmit
from bokeh.models.callbacks import CustomJS
from bokeh.layouts import column, row
from bokeh.events import ButtonClick
import random
import os
import json
import numpy as np

TO_IGNORE= ['168b7587-f0e6-47b2-bb9b-831840f7eb8b_32351.mp4']
def check(clip_f):
    if clip_f[:clip_f.rfind(".")] in TO_IGNORE:
        return False
    return True

f = open('data_jsons/name_to_link.json')
name_to_link = json.load(f)

f = open('data_jsons/clip_to_question.json')
clip_to_question = json.load(f)

AMOUNT = 500
CIRCLE_SIZE = 40
IMAGE_SIZE = 36


x = random.sample(list(range(600)), AMOUNT)
y = np.array(random.sample(list(range(600)), AMOUNT)) / 2
clip_names = [clip_f[:clip_f.rfind(".")] for clip_f in os.listdir("transparent_images") if check(clip_f)][:AMOUNT]
frame_locations = [f"transparent_images/{clip_n}.png" for clip_n in clip_names]
original_frame_locations = [f"images/{clip_n}.jpg" for clip_n in clip_names]
clip_url = [name_to_link[f"{n}.mp4"] for n in clip_names]
clip_q = []
clip_a = []
clip_wa = []
clip_wb = []
clip_wc = []
clip_wd = []

for clip_n in clip_names:
    qa_string = clip_to_question[clip_n]
    qaw = qa_string.split("\n")
    q = qaw[0][qaw[0].find(":") + 2:]
    a = qaw[1][qaw[1].find(":") + 2:]
    wa = qaw[2][qaw[2].find(":") + 2:]
    wb = qaw[3][qaw[3].find(":") + 2:]
    wc = qaw[4][qaw[4].find(":") + 2:]
    wd = qaw[5][qaw[5].find(":") + 2:]
    
    clip_q.append(q)
    clip_a.append(a)
    clip_wa.append(wa)
    clip_wb.append(wb)
    clip_wc.append(wc)
    clip_wd.append(wd)
    
data = {'x':x,
        'y':y,
        'clip_names':clip_names,
        'url':frame_locations,
        'original_url':original_frame_locations,
        'clip_url':clip_url,
        'q':clip_q,
        'a':clip_a,
        'wa':clip_wa,
        'wb':clip_wb,
        'wc':clip_wc,
        'wd':clip_wd,
        'alpha':[1]*AMOUNT,
        }


source = ColumnDataSource(data)

f = open("html_txts/hover.txt", "r")
HOWER = f.read()

f = open("html_txts/js_code.txt", "r")
JS_CODE = f.read()

f = open("html_txts/search_code.txt", "r")
SEARCH_CODE = f.read()

p = figure(sizing_mode="stretch_both", 
           width=2500, 
           height=700, 
           tooltips=HOWER, 
           tools="pan,wheel_zoom,reset,tap,hover",)
p.outline_line_width = 3
p.outline_line_alpha = 0.5
p.outline_line_color = "navy"

p.xaxis.visible = False
p.yaxis.visible = False
p.xgrid.visible = False
p.ygrid.visible = False

circle = p.circle(x = 'x', y='y', fill_color="red", size=CIRCLE_SIZE, alpha='alpha', source = source,)
images = p.image_url(url = 'url', x = 'x', y='y',
                     anchor="center", w=IMAGE_SIZE, w_units="screen", 
                     h=IMAGE_SIZE, h_units="screen", dilate=True, alpha='alpha',
                     source = source, )

callback = CustomJS(args=dict(source=source), code=JS_CODE)
search_callback = CustomJS(args=dict(source=source, renderer=images), code=SEARCH_CODE)

source.selected.js_on_change('indices', callback)
p.hover.renderers = [circle]

text_input = TextInput(placeholder="Input a value and press Enter ...", width=1500, height = 50, css_classes = ["text-input-1223"])
text_input.sizing_mode = "stretch_width"
text_input.js_on_event(ValueSubmit, search_callback)

layout = column(p, text_input)
layout.sizing_mode = "stretch_both"

# save the results to a file
output_file(filename="custom_filename.html", title="Static HTML file")
show(layout)

In [149]:
f = open('name_to_link.json')
name_to_link = json.load(f)

In [150]:
x = random.sample(list(range(200)), 50)
y = random.sample(list(range(100)), 50)
clip_names = [clip_f[:clip_f.rfind(".")] for clip_f in os.listdir("transparent_images")][:50]
frame_locations = [f"transparent_images/{clip_n}.png" for clip_n in clip_names]
clip_urls = [name_to_link[f"{n}.mp4"] for n in clip_names]

In [159]:
source = ColumnDataSource(data=dict(
    x=x,
    y=y,
    clip_names=clip_names,
    url=frame_locations,
    clip_urls=clip_urls
))

In [160]:
f = open("hover.txt", "r")
HOWER = f.read()

In [161]:
callback = CustomJS(args=dict(source=source), code="""
window.alert(source.selected.indices);
var myIndex = source.selected.indices;
var myUrl = source.data.clip_urls[myIndex];
window.alert(myUrl);
window.location.href = "file:///C:/Users/artma/Desktop/School/website/clip.html?param1=" + encodeURIComponent(myUrl);;
""")

In [162]:
p = figure(sizing_mode="stretch_width", max_width=1100, height=600, tooltips=HOWER, tools="pan,wheel_zoom,reset,tap,hover",)

# add a circle renderer
circle = p.circle(source = source, fill_color="red", size=70, )
images = p.image_url(source = source, anchor="center", 
            w=67, w_units="screen", h=67, h_units="screen", dilate=True)
#taptool = p.select(type=TapTool)
#taptool.callback = callback
source.selected.js_on_change('indices', callback)
p.hover.renderers = [circle]

# save the results to a file
output_file(filename="custom_filename.html", title="Static HTML file")
show(p)

In [10]:
from bokeh.models import ColumnDataSource, OpenURL, TapTool
from bokeh.plotting import figure, output_file, save, show
from bokeh.events import Tap
from bokeh.models.callbacks import CustomJS

#Value: 'string_of_characters_like_these%3A%24%23%40%3D%3F%25%5EQ%5E%24'


source = ColumnDataSource(data=dict(
    x=[1, 2, 3, 4, 5],
    y=[2, 5, 8, 2, 7],
    desc=['A', 'b', 'C', 'd', 'E'],
    url=[
        "transparent_images/023bf95e-28de-43b4-a43f-720edba667a5_51928.png",
        "transparent_images/023bf95e-28de-43b4-a43f-720edba667a5_51928.png",
        "transparent_images/023bf95e-28de-43b4-a43f-720edba667a5_51928.png",
        "transparent_images/023bf95e-28de-43b4-a43f-720edba667a5_51928.png",
        "transparent_images/023bf95e-28de-43b4-a43f-720edba667a5_51928.png"
    ],
))

f = open("hover.txt", "r")
HOWER = f.read()

callback = CustomJS(args=dict(source=source), code="""
var myUrl = "https://drive.google.com/uc?export=view&id=1PqERbEceLHiO1bhBxqWxwuCUWp7XxAKt";
window.location.href = "file:///C:/Users/artma/Desktop/School/website/clip.html?param1=" + encodeURIComponent(myUrl);;
""")


p = figure(sizing_mode="stretch_width", max_width=500, height=500, tooltips=HOWER, tools="pan,wheel_zoom,reset,tap,hover",)

# add a circle renderer
circle = p.circle(source = source, fill_color="red", size=70, )
images = p.image_url(source = source, anchor="center", 
            w=45, w_units="screen", h=45, h_units="screen", dilate=True)
taptool = p.select(type=TapTool)
taptool.callback = callback
p.hover.renderers = [circle]

# save the results to a file
output_file(filename="custom_filename.html", title="Static HTML file")
show(p)